In [ ]:
from sqlalchemy import text, create_engine
import pandas as pd
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 
import os
import numpy as np 

In [ ]:
# Importing environmental variables library that reads from the .env file
from dotenv import load_dotenv

# Loading key-value pairs from the .env file into the OS environment
load_dotenv()

# Reading the key-value pairs from the OS environment
user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")
db_hostname = os.getenv("DB_HOST")
db_name = os.getenv("DB_DATABASE")

# Using those variables in the connection string using "F" strings
conn_string = f"mysql+mysqldb://{user}:{password}@{db_hostname}/{db_name}?charset=utf8mb4"
engine = create_engine(conn_string)

In [ ]:
consultant_name_df = pd.read_excel('G:/Post Construction/metadata csvs/consultants.xlsx', encoding = 'utf-8-sig')
consultant_name_lst = consultant_name_df["consultant_name"].tolist()

In [ ]:
with engine.connect() as conn: ## with context manager with, we don't have to close the connection everytime we run queries
    query1 = "SELECT * FROM pdfs;"
    df1 = pd.read_sql(query1, conn)

In [ ]:
main_consultant_name = []
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\t1nipun\AppData\Local\Tesseract-OCR\tesseract.exe'
for file in df1['pdfName']:
    pages = convert_from_path("G:/Post Construction/PCMR_All/" + file + '.pdf', 500, last_page = 1)
    for page in pages:
        filename = file +'_Page1.jpg'
        page.save("C:/Users/t1nipun/Desktop/PCMR/human-robot/Data_Analysis/pdf_firstpage_img/" + filename, 'JPEG')
        text = str(((pytesseract.image_to_string(Image.open("C:/Users/t1nipun/Desktop/PCMR/human-robot/Data_Analysis/pdf_firstpage_img/" + filename))))).lower()
        consultant_name = []
        for name in consultant_name_lst:
            if name.lower() in text:
                consultant_name.append(name)
        main_consultant_name.append(consultant_name)

In [ ]:
df1['consultant_name'] = main_consultant_name
mapping_dict = df1.groupby('filingId')['consultant_name'].apply(lambda x: x.values.tolist()).to_dict()

In [173]:
def flatten(lst_of_lsts):
    output = []
    for element in lst_of_lsts:
        if type(element) == list:
            output.extend(flatten(element))
        else:
            output.append(element)
    return output

In [178]:
newdict = {key: flatten(value) for key, value in mapping_dict.items()}
newdict

{337440: ['AXYS Environmental Consulting Ltd'],
 340412: [],
 399962: [],
 409162: ['AXYS Environmental Consulting Ltd'],
 476698: [],
 485676: ['TERA Environmental Consultants'],
 485763: ['TERA Environmental Consultants'],
 495396: [],
 538744: ['TERA Environmental Consultants'],
 538754: ['TERA Environmental Consultants'],
 541474: [],
 542503: [],
 542626: [],
 543503: ['TERA Environmental Consultants'],
 546497: ['TERA Environmental Consultants', 'TERA Environmental Consultants'],
 550966: ['TERA Environmental Consultants', 'Applied Aquatic Research Ltd'],
 563853: [],
 579440: ['TERA Environmental Consultants'],
 582277: ['TERA Environmental Consultants'],
 587008: [],
 587052: [],
 587537: ['TERA Environmental Consultants', 'B.A. Blackwell & Associates Ltd'],
 588202: ['Golder Associates'],
 590444: [],
 590454: [],
 594990: [],
 601828: [],
 659952: ['Golder Associates'],
 664701: ['TERA Environmental Consultants'],
 664824: ['TERA Environmental Consultants'],
 665691: ['TERA E

In [ ]:
# columns  = ['pdfId', 'pdfName', 'filingId', 'totalPages', 'application_id', 'submitter', 'company', 'consultant_name']
# df1.to_csv('consultant_name_analysis.csv', encoding = 'utf-8-sig', columns = columns)

In [177]:
newdict[779235]

['TERA Environmental Consultants',
 'B.A. Blackwell and Associates',
 'Applied Geomarine Environmental Consultants Ltd',
 'Westland Resource Group']